

![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://githubtocolab.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/streamlit_notebooks/SENTIMENT_EN_CYBERBULLYING.ipynb)




# **Detect bullying in tweets**

## 1. Colab Setup

In [1]:
!wget http://setup.johnsnowlabs.com/colab.sh -O - | bash
# !bash colab.sh
# -p is for pyspark
# -s is for spark-nlp
# !bash colab.sh -p 3.1.1 -s 3.0.1
# by default they are set to the latest

openjdk version "11.0.10" 2021-01-19
OpenJDK Runtime Environment (build 11.0.10+9-Ubuntu-0ubuntu1.18.04)
OpenJDK 64-Bit Server VM (build 11.0.10+9-Ubuntu-0ubuntu1.18.04, mixed mode, sharing)
setup Colab for PySpark 3.1.1 and Spark NLP 3.0.0
     |████████████████████████████████| 212.3MB 58kB/s 
     |████████████████████████████████| 143kB 45.4MB/s 
     |████████████████████████████████| 204kB 47.0MB/s 


In [2]:
import pandas as pd
import numpy as np
import json
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from sparknlp.annotator import *
from sparknlp.base import *
import sparknlp
from sparknlp.pretrained import PretrainedPipeline

## 2. Start Spark Session

In [3]:
spark = sparknlp.start()

## 3. Select the DL model

In [4]:
MODEL_NAME='classifierdl_use_cyberbullying'

## 4. Some sample examples

In [5]:
## Generating Example Files ##

text_list = [
             #neutral
             "@CALMicC he kept me informed on stuff id missed and seemed ok. I liked him.",
             #sexism
             "@AMohedin Okay, we have women being physically inferior and the either emotionally or mentally inferior in some way.",
             #neutral
             "@LynnMagic people think that implying association via follow is a bad thing. but it's shockingly accurate.",
             #racism
             "@Rayandawlah_ @_Jihad10 These days might and honor come from science, technology, humanitarianism. Which is why Muslims won't get any.",
             #sexism
             "Stay outve Congress and we have a deal. @jacobkramer17 Call me sexist bt the super bowl should b guys only no women are allowed n th stadium",
             #neutral
             "I'm looking for a few people to help with @ggautoblocker's twitter. Log &amp; categorize mentions as support requests/abusive/positive tweets.",
             #racism
             "@geeky_zekey Thanks for showing again that blacks are the biggest racists. Blocked",
             #sexism
             """@ListenToRaisin No question. Feminists have the media. Did you see any mention of Clem Fords OPEN bigotry, etc?  Nope. "Narrative" is all.""",
             #sexism
             "RT @EBeisner @ahall012 I agree with you!! I would rather brush my teeth with sandpaper then watch football with a girl!!", 
             #racism
             "@hibach8 But it is a lie.  The religion is a disgusting, terrorist, hate mongering piece of filth.  That has nothing to do with individuals.",
  ]


## 5. Define Spark NLP pipeline

In [6]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")
    
use = UniversalSentenceEncoder.pretrained(name="tfhub_use", lang="en")\
 .setInputCols(["document"])\
 .setOutputCol("sentence_embeddings")


sentimentdl = ClassifierDLModel.pretrained(name=MODEL_NAME)\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("sentiment")

nlpPipeline = Pipeline(
      stages = [
          documentAssembler,
          use,
          sentimentdl
      ])


tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]
classifierdl_use_cyberbullying download started this may take some time.
Approximate size to download 21.3 MB
[OK!]


## 6. Run the pipeline

In [7]:
empty_df = spark.createDataFrame([['']]).toDF("text")
pipelineModel = nlpPipeline.fit(empty_df)
df = spark.createDataFrame(pd.DataFrame({"text":text_list}))
result = pipelineModel.transform(df)

## 7. Visualize results

In [8]:
result.select(F.explode(F.arrays_zip('document.result', 'sentiment.result')).alias("cols")) \
.select(F.expr("cols['0']").alias("document"),
        F.expr("cols['1']").alias("sentiment")).show(truncate=False)

+----------------------------------------------------------------------------------------------------------------------------------------------+---------+
|document                                                                                                                                      |sentiment|
+----------------------------------------------------------------------------------------------------------------------------------------------+---------+
|@CALMicC he kept me informed on stuff id missed and seemed ok. I liked him.                                                                   |neutral  |
|@AMohedin Okay, we have women being physically inferior and the either emotionally or mentally inferior in some way.                          |sexism   |
|@LynnMagic people think that implying association via follow is a bad thing. but it's shockingly accurate.                                    |neutral  |
|@Rayandawlah_ @_Jihad10 These days might and honor come from science,